In [23]:
import numpy as np
import pandas as pd

In [343]:
def latlng2_manhattan_distance(loc1, loc2):

    lat_lon_1 = np.radians(loc1)
    lat_lon_2 = np.radians(loc2)
    d_lat_lon = np.abs(lat_lon_1- lat_lon_2)
    r = 6373.0
    a_lat_lon = np.sin(d_lat_lon / 2.0) **2
    c = 2 * np.arctan2(np.sqrt(a_lat_lon), np.sqrt(1 - a_lat_lon))
    c = r * c
    c = np.sum(c, axis=1)
    
    return c

In [449]:
#日期，业务种类，重量，揽投机构，道段(揽收只有业务员信息，没有道段故不收入)
print("0")
#读取揽收数据
data_lanshou = pd.read_csv('江门揽收数据.csv', encoding='gb18030', usecols=[0,1,3,6,8])
data_lanshou = data_lanshou[['D_CLCT_DATE','V_MAIL_CODE','V_BUSI_TYPE', 'F_WEIGH_WEIGHT', 'V_CLCT_ORG_CODE']]#调整一下顺序
data_lanshou['D_CLCT_DATE'] = pd.to_datetime(data_lanshou['D_CLCT_DATE'])
#加入经纬度
data_lanshou_ = pd.read_csv('coord/lanshou.csv', encoding='gb18030', usecols=[1,2])
data_lanshou['lng'] = data_lanshou_['lng']
data_lanshou['lat'] = data_lanshou_['lat']
#揽收默认的列属性不太一样，改成与投递一致
new_cols = ["投递日期","邮件号","业务种类","重量","投递机构","lng","lat"]
data_lanshou.columns = new_cols
data_lanshou['邮件号'].apply(str)
print("1")
#读取投递数据1
data_toudi21 = pd.read_csv('toudi_21.csv', encoding='gb18030', usecols=[0,1,2,3,4,7])
data_toudi21['道段'] = data_toudi21['道段'].str.split(' ', expand=True)[0]
data_toudi21['投递日期'] = pd.to_datetime(data_toudi21['投递日期'])
#加入经纬度
data_toudi21_  = pd.read_csv('coord/toudi_21.csv', encoding='gb18030', usecols=[1,2])
data_toudi21['lng'] = data_toudi21_['lng']
data_toudi21['lat'] = data_toudi21_['lat']
print("2")
#读取投递数据2
data_toudi22 = pd.read_csv('toudi_22.csv', encoding='gb18030', usecols=[0,1,2,3,4,7])
data_toudi22['道段'] = data_toudi22['道段'].str.split(' ', expand=True)[0]
data_toudi22['投递日期'] = pd.to_datetime(data_toudi22['投递日期'])
#加入经纬度
data_toudi22_  = pd.read_csv('coord/toudi_21.csv', encoding='gb18030', usecols=[1,2])
data_toudi22['lng'] = data_toudi22_['lng']
data_toudi22['lat'] = data_toudi22_['lat']
print("3")
#读取投递数据3
data_toudi206 = pd.read_csv('toudi_206.csv', encoding='gb18030', usecols=[0,1,2,3,4,7])
data_toudi206['道段'] = data_toudi206['道段'].str.split(' ', expand=True)[0]
data_toudi206['投递日期'] = pd.to_datetime(data_toudi206['投递日期'])
#加入经纬度
data_toudi206_  = pd.read_csv('coord/toudi_21.csv', encoding='gb18030', usecols=[1,2])
data_toudi206['lng'] = data_toudi206_['lng']
data_toudi206['lat'] = data_toudi206_['lat']
print("4")

0
1
2
3
4


In [450]:
#将不存在的部门排除
departments = pd.read_excel('江门补充资料/江门机构.xlsx', usecols=[3,11,12])
temp = departments['机构代码'].tolist()

# data_lanshou
bools = []
for row in data_lanshou.values:
    bools.append(row[4] in temp)
data_lanshou = data_lanshou.loc[bools]

# data_toudi21
bools = []
for row in data_toudi21.values:
    bools.append(row[4] in temp)
data_toudi21 = data_toudi21.loc[bools]

# data_toudi22
bools = []
for row in data_toudi22.values:
    bools.append(row[4] in temp)
data_toudi22 = data_toudi22.loc[bools]

# data_toudi206
bools = []
for row in data_toudi206.values:
    bools.append(row[4] in temp)
data_toudi206 = data_toudi206.loc[bools]

In [451]:
#exclude为不需要的日期列表，下述代码将周末的数据排除
exclude = ["2020-08-01","2020-08-02","2020-08-08","2020-08-09","2020-08-15","2020-08-16","2020-08-22","2020-08-23","2020-08-29","2020-08-30"]

bools = []
for row in data_lanshou.values:
    bools.append(str(row[0].date()) not in exclude)
data_lanshou = data_lanshou.loc[bools]

bools = []
for row in data_toudi21.values:
    bools.append(str(row[0].date()) not in exclude)
data_toudi21 = data_toudi21.loc[bools]

bools = []
for row in data_toudi22.values:
    bools.append(str(row[0].date()) not in exclude)
data_toudi22 = data_toudi22.loc[bools]
    
bools = []
for row in data_toudi206.values:
    bools.append(str(row[0].date()) not in exclude)
data_toudi206 = data_toudi206.loc[bools]

In [452]:
#重量归一化
#求所有揽收数据的重量均值和标准差
temp = data_lanshou.loc[:,'重量']
mean = temp.mean()
std = temp.std()
temp = temp.map(lambda x: 50+10*(x-mean)/std)
data_lanshou.loc[:,'重量'] = temp

#求所有投递数据的重量均值和标准差
temp = data_toudi21.loc[:,'重量'].values
print(len(temp))
temp = np.insert(temp,1,data_toudi22.loc[:,'重量'].values)
print(len(temp))
temp = np.insert(temp,1,data_toudi206.loc[:,'重量'].values)
print(len(temp))
mean = np.nanmean(temp)
std = np.nanstd(temp)

data_toudi21['重量'].fillna(mean, inplace=True)
data_toudi22['重量'].fillna(mean, inplace=True)
data_toudi206['重量'].fillna(mean, inplace=True)

temp = data_toudi21['重量']
temp = temp.map(lambda x: 50+10*(x-mean)/std)
data_toudi21.loc[:,'重量'] = temp

temp = data_toudi22['重量']
temp = temp.map(lambda x: 50+10*(x-mean)/std)
data_toudi22.loc[:,'重量'] = temp

temp = data_toudi206['重量']
temp = temp.map(lambda x: 50+10*(x-mean)/std)
data_toudi206.loc[:,'重量'] = temp

558886
1151181
1430160


In [453]:
#去除外省数据以及缺失数据，此处直接将经纬度大的离谱的和nan去除
temp = data_lanshou.loc[data_lanshou['lng'].isnull().values.tolist()]
data_lanshou = data_lanshou.drop(temp.index, axis=0)
temp = data_lanshou.loc[(data_lanshou['lng']>114) | (data_lanshou['lng']<111)]
data_lanshou = data_lanshou.drop(temp.index)
temp = data_lanshou.loc[(data_lanshou['lat']>23) | (data_lanshou['lat']<21)]
data_lanshou = data_lanshou.drop(temp.index)

temp = data_toudi21.loc[data_toudi21['lng'].isnull().values.tolist()]
data_toudi21 = data_toudi21.drop(temp.index, axis=0)
temp = data_toudi21.loc[(data_toudi21['lng']>114) | (data_toudi21['lng']<111)]
data_toudi21 = data_toudi21.drop(temp.index)
temp = data_toudi21.loc[(data_toudi21['lat']>23) | (data_toudi21['lat']<21)]
data_toudi21 = data_toudi21.drop(temp.index)

temp = data_toudi22.loc[data_toudi22['lng'].isnull().values.tolist()]
data_toudi22 = data_toudi22.drop(temp.index, axis=0)
temp = data_toudi22.loc[(data_toudi22['lng']>114) | (data_toudi22['lng']<111)]
data_toudi22 = data_toudi22.drop(temp.index)
temp = data_toudi22.loc[(data_toudi22['lat']>23) | (data_toudi22['lat']<21)]
data_toudi22 = data_toudi22.drop(temp.index)

temp = data_toudi206.loc[data_toudi206['lng'].isnull().values.tolist()]
data_toudi206 = data_toudi206.drop(temp.index, axis=0)
temp = data_toudi206.loc[(data_toudi206['lng']>114) | (data_toudi206['lng']<111)]
data_toudi206 = data_toudi206.drop(temp.index)
temp = data_toudi206.loc[(data_toudi206['lat']>23) | (data_toudi206['lat']<21)]
data_toudi206 = data_toudi206.drop(temp.index)

In [454]:
departments = pd.read_excel('江门补充资料/江门机构.xlsx', usecols=[3,11,12])
#计算距离
data_lanshou['dist'] = ''
for i in range(len(departments)):
    temp = data_lanshou.loc[data_lanshou['投递机构'] == departments.loc[i,:][0], ['lat','lng']]
    dist = latlng2_manhattan_distance(np.array(departments.loc[i,:][1:3]), np.array(temp))
    data_lanshou.loc[data_lanshou['投递机构'] == departments.loc[i,:][0], 'dist'] = dist
    
data_toudi21['dist'] = ''
for i in range(len(departments)):
    temp = data_toudi21.loc[data_toudi21['投递机构'] == departments.loc[i,:][0], ['lat','lng']]
    dist = latlng2_manhattan_distance(np.array(departments.loc[i,:][1:3]), np.array(temp))
    data_toudi21.loc[data_toudi21['投递机构'] == departments.loc[i,:][0], 'dist'] = dist
    
data_toudi22['dist'] = ''
for i in range(len(departments)):
    temp = data_toudi22.loc[data_toudi22['投递机构'] == departments.loc[i,:][0], ['lat','lng']]
    dist = latlng2_manhattan_distance(np.array(departments.loc[i,:][1:3]), np.array(temp))
    data_toudi22.loc[data_toudi22['投递机构'] == departments.loc[i,:][0], 'dist'] = dist
    
data_toudi206['dist'] = ''
for i in range(len(departments)):
    temp = data_toudi206.loc[data_toudi206['投递机构'] == departments.loc[i,:][0], ['lat','lng']]
    dist = latlng2_manhattan_distance(np.array(departments.loc[i,:][1:3]), np.array(temp))
    data_toudi206.loc[data_toudi206['投递机构'] == departments.loc[i,:][0], 'dist'] = dist

In [511]:
#代收和高端业务增加难度系数

#寻找快包
bools = data_lanshou['邮件号'].str.contains("^1.*8[0-9]$",na=False)
data_lanshou.loc[bools,'业务种类'] = "代收"

bools = data_toudi21['邮件号'].str.contains("^1.*8[0-9]$",na=False)
data_toudi21.loc[bools,'业务种类'] = "代收"

bools = data_toudi22['邮件号'].str.contains("^1.*8[0-9]$",na=False)
data_toudi22.loc[bools,'业务种类'] = "代收"

bools = data_toudi206['邮件号'].str.contains("^1.*8[0-9]$",na=False)
data_toudi206.loc[bools,'业务种类'] = "代收"

#设置代价
data_lanshou.loc[data_lanshou['业务种类'] == "代收", "业务种类"] = 1.2
data_lanshou.loc[data_lanshou['业务种类'] == "高端政务", "业务种类"] = 1.2
data_lanshou.loc[data_lanshou['业务种类'] != 1.2, "业务种类"] = 1

data_toudi21.loc[data_toudi21['业务种类'] == "代收", "业务种类"] = 1.2
data_toudi21.loc[data_toudi21['业务种类'] == "高端政务", "业务种类"] = 1.2
data_toudi21.loc[data_toudi21['业务种类'] != 1.2, "业务种类"] = 1

data_toudi22.loc[data_toudi22['业务种类'] == "代收", "业务种类"] = 1.2
data_toudi22.loc[data_toudi22['业务种类'] == "高端政务", "业务种类"] = 1.2
data_toudi22.loc[data_toudi22['业务种类'] != 1.2, "业务种类"] = 1

data_toudi206.loc[data_toudi206['业务种类'] == "代收", "业务种类"] = 1.2
data_toudi206.loc[data_toudi206['业务种类'] == "高端政务", "业务种类"] = 1.2
data_toudi206.loc[data_toudi206['业务种类'] != 1.2, "业务种类"] = 1

In [599]:
#按部门分开存储，放置于列表中
lanshou = []
toudi21 = []
toudi22 = []
toudi206 = []

departments = pd.read_excel('江门补充资料/江门机构.xlsx', usecols=[3])
for department in departments.values:
    
    batch = data_lanshou.loc[data_lanshou['投递机构']==int(department)]
    lanshou.append(batch)
    batch = data_toudi21.loc[data_toudi21['投递机构']==int(department)]
    toudi21.append(batch)
    batch = data_toudi22.loc[data_toudi22['投递机构']==int(department)]
    toudi22.append(batch)
    batch = data_toudi206.loc[data_toudi206['投递机构']==int(department)]
    toudi206.append(batch)

In [635]:
dates = ["2020-08-03 00:00:00", "2020-08-03 12:00:00", "2020-08-04 00:00:00", "2020-08-04 12:00:00", "2020-08-05 00:00:00",
         "2020-08-05 12:00:00", "2020-08-06 00:00:00", "2020-08-06 12:00:00", "2020-08-07 00:00:00", "2020-08-07 12:00:00",
         "2020-08-12 00:00:00", "2020-08-12 12:00:00", "2020-08-13 00:00:00", "2020-08-13 12:00:00", "2020-08-14 00:00:00",
         "2020-08-14 12:00:00", "2020-08-17 00:00:00", "2020-08-17 12:00:00", "2020-08-18 00:00:00", "2020-08-18 12:00:00",
         "2020-08-19 00:00:00", "2020-08-19 12:00:00", "2020-08-20 00:00:00", "2020-08-20 12:00:00", "2020-08-21 00:00:00",
         "2020-08-21 12:00:00", "2020-08-24 00:00:00", "2020-08-24 12:00:00", "2020-08-25 00:00:00", "2020-08-25 12:00:00",
         "2020-08-26 00:00:00", "2020-08-26 12:00:00", "2020-08-27 00:00:00", "2020-08-27 12:00:00", "2020-08-28 00:00:00",
         "2020-08-28 12:00:00", "2020-08-31 00:00:00", "2020-08-31 12:00:00", "2020-09-01 00:00:00"]

costs_lanshou = []#存放揽收各部门cost
for department in lanshou:
    cost = 0
    for i in range(0,len(dates)-1):
        #筛选日期
        temp = department.loc[(department['投递日期'] > dates[i]) & (department['投递日期'] < dates[i+1])]
        #计数同地点同时间的记录并去除重复
        count = temp[['lng', 'lat', 'dist']].groupby(['lng', 'lat']).count()['dist'].values
        temp = temp.drop_duplicates(subset=['lng','lat'],keep='first',inplace=False)
        #计算
        dist = np.array(temp['dist'], dtype=np.float32)
        weight = np.array(temp['重量'], dtype=np.float32)
        form = np.array(temp['业务种类'], dtype=np.float32)
        assert(count.shape == dist.shape)
        cost += np.sum(np.sqrt(count) * np.log2(dist) * weight * form, axis=0)
    costs_lanshou.append(cost)

costs_toudi21 = []#存放揽收各部门cost
for department in toudi21:
    cost = 0
    for i in range(0,len(dates)-1):
        #筛选日期
        temp = department.loc[(department['投递日期'] > dates[i]) & (department['投递日期'] < dates[i+1])]
        #计数同地点同时间的记录并去除重复
        count = temp[['lng', 'lat', 'dist']].groupby(['lng', 'lat']).count()['dist'].values
        temp = temp.drop_duplicates(subset=['lng','lat'],keep='first',inplace=False)
        #计算
        dist = np.array(temp['dist'], dtype=np.float32)
        weight = np.array(temp['重量'], dtype=np.float32)
        form = np.array(temp['业务种类'], dtype=np.float32)
        assert(count.shape == dist.shape)
        cost += np.sum(np.sqrt(count) * np.log2(dist) * weight * form, axis=0)
    costs_toudi21.append(cost)
    
costs_toudi22 = []#存放揽收各部门cost
for department in toudi22:
    cost = 0
    for i in range(0,len(dates)-1):
        #筛选日期
        temp = department.loc[(department['投递日期'] > dates[i]) & (department['投递日期'] < dates[i+1])]
        #计数同地点同时间的记录并去除重复
        count = temp[['lng', 'lat', 'dist']].groupby(['lng', 'lat']).count()['dist'].values
        temp = temp.drop_duplicates(subset=['lng','lat'],keep='first',inplace=False)
        #计算
        dist = np.array(temp['dist'], dtype=np.float32)
        weight = np.array(temp['重量'], dtype=np.float32)
        form = np.array(temp['业务种类'], dtype=np.float32)
        assert(count.shape == dist.shape)
        cost += np.sum(np.sqrt(count) * np.log2(dist) * weight * form, axis=0)
    costs_toudi22.append(cost)
    
costs_toudi206 = []#存放揽收各部门cost
for department in toudi206:
    cost = 0
    for i in range(0,len(dates)-1):
        #筛选日期
        temp = department.loc[(department['投递日期'] > dates[i]) & (department['投递日期'] < dates[i+1])]
        #计数同地点同时间的记录并去除重复
        count = temp[['lng', 'lat', 'dist']].groupby(['lng', 'lat']).count()['dist'].values
        temp = temp.drop_duplicates(subset=['lng','lat'],keep='first',inplace=False)
        #计算
        dist = np.array(temp['dist'], dtype=np.float32)
        weight = np.array(temp['重量'], dtype=np.float32)
        form = np.array(temp['业务种类'], dtype=np.float32)
        assert(count.shape == dist.shape)
        cost += np.sum(np.sqrt(count) * np.log2(dist) * weight * form, axis=0)
    costs_toudi206.append(cost)

In [643]:
costs = np.array(costs_lanshou) + np.array(costs_toudi21) + np.array(costs_toudi22) + np.array(costs_toudi206)

In [652]:
print(np.array(costs_toudi206).sum())
print(np.array(costs_toudi22).sum())
print(np.array(costs_toudi21).sum())
print(np.array(costs_lanshou).sum())

64850225.1129673
129942716.56274758
29748458.267988708
5562351.298941471


In [656]:
len(data_lanshou)

171925

### 以下代码暂时废了

In [155]:
#读取投递数据1
data_toudi21 = pd.read_csv('toudi_21.csv', encoding='gb18030', usecols=[0,1,2,3,4,7,8,10])
data_toudi21['道段'] = data_toudi21['道段'].str.split(' ', expand=True)[0]
data_toudi21['投递日期'] = pd.to_datetime(data_toudi21['投递日期'])

data_toudi21 = data_toudi21.loc[data_toudi21['退回妥投_收件人地址'].isnull().values.tolist()]

ss= data_toudi21[['投递地址','投递日期']]
ss['投递地址'] = ss['投递地址'].apply(str)
bool = ss['投递地址'].str.contains('四川省').values
ss = ss[bool]
data_toudi21 = data_toudi21.loc[ss.index,:]


writer = pd.ExcelWriter('output.xlsx')
data_toudi21.to_excel(writer)
writer.save()

<ipython-input-155-5b55da4b2e93>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss['投递地址'] = ss['投递地址'].apply(str)


In [18]:
#暂时不用
#揽收数据没有道段，故不划分道段

#对toudi21的每个机构进行处理
for department in toudi21:
    department_temp = department.loc[department['lng'].isnull().values.tolist()] #不含经纬度的数据
    department_done = department.drop(department_temp.index)
    roads = department_temp['道段'].unique()
    roads = roads[~pd.isnull(roads)]
    #对某个机构中的每个道段进行处理
    for road in roads:
        data_road = department_done.loc[department_done['道段']==road]
        maxy = data_road['lng'].max()
        miny = data_road['lng'].min()
        maxx = data_road['lat'].max()
        minx = data_road['lat'].min()
        temp_road = department_temp.loc[department_temp['道段']==road]
        lng = np.random.random(len(temp_road)) * (maxy - miny) + miny
        lat = np.random.random(len(temp_road)) * (maxx - minx) + minx
        department_temp.loc[department_temp['道段']==road, 'lng'] = lng
        department_temp.loc[department_temp['道段']==road, 'lat'] = lat
    #对没有道段数据的记录，即nan值做特殊处理(取处理的最后一个道段的随机经纬度)
    nan = department_temp.loc[department_temp['道段'].isnull().values.tolist()]
    if len(nan) > 0:
        lng = np.random.random(len(nan)) * (maxy - miny) + miny
        lat = np.random.random(len(nan)) * (maxx - minx) + minx
        department_temp.loc[nan.index, 'lng'] = lng
        department_temp.loc[nan.index, 'lat'] = lat
    
    department.loc[department['lng'].isnull().values.tolist(), 'lng'] = department_temp['lng']
    department.loc[department['lat'].isnull().values.tolist(), 'lat'] = department_temp['lat']
    
#对toudi22的每个机构进行处理
for department in toudi22:
    department_temp = department.loc[department['lng'].isnull().values.tolist()] #不含经纬度的数据
    department_done = department.drop(department_temp.index)
    roads = department_temp['道段'].unique()
    roads = roads[~pd.isnull(roads)]
    #对某个机构中的每个道段进行处理
    for road in roads:
        data_road = department_done.loc[department_done['道段']==road]
        maxy = data_road['lng'].max()
        miny = data_road['lng'].min()
        maxx = data_road['lat'].max()
        minx = data_road['lat'].min()
        temp_road = department_temp.loc[department_temp['道段']==road]
        lng = np.random.random(len(temp_road)) * (maxy - miny) + miny
        lat = np.random.random(len(temp_road)) * (maxx - minx) + minx
        department_temp.loc[department_temp['道段']==road, 'lng'] = lng
        department_temp.loc[department_temp['道段']==road, 'lat'] = lat
    #对没有道段数据的记录，即nan值做特殊处理(取处理的最后一个道段的随机经纬度)
    nan = department_temp.loc[department_temp['道段'].isnull().values.tolist()]
    if len(nan) > 0:
        lng = np.random.random(len(nan)) * (maxy - miny) + miny
        lat = np.random.random(len(nan)) * (maxx - minx) + minx
        department_temp.loc[nan.index, 'lng'] = lng
        department_temp.loc[nan.index, 'lat'] = lat
    
    department.loc[department['lng'].isnull().values.tolist(), 'lng'] = department_temp['lng']
    department.loc[department['lat'].isnull().values.tolist(), 'lat'] = department_temp['lat']
    
#对toudi206的每个机构进行处理
for department in toudi206:
    department_temp = department.loc[department['lng'].isnull().values.tolist()] #不含经纬度的数据
    department_done = department.drop(department_temp.index)
    roads = department_temp['道段'].unique()
    roads = roads[~pd.isnull(roads)]
    #对某个机构中的每个道段进行处理
    for road in roads:
        data_road = department_done.loc[department_done['道段']==road]
        maxy = data_road['lng'].max()
        miny = data_road['lng'].min()
        maxx = data_road['lat'].max()
        minx = data_road['lat'].min()
        temp_road = department_temp.loc[department_temp['道段']==road]
        lng = np.random.random(len(temp_road)) * (maxy - miny) + miny
        lat = np.random.random(len(temp_road)) * (maxx - minx) + minx
        department_temp.loc[department_temp['道段']==road, 'lng'] = lng
        department_temp.loc[department_temp['道段']==road, 'lat'] = lat
    #对没有道段数据的记录，即nan值做特殊处理(取处理的最后一个道段的随机经纬度)
    nan = department_temp.loc[department_temp['道段'].isnull().values.tolist()]
    if len(nan) > 0:
        lng = np.random.random(len(nan)) * (maxy - miny) + miny
        lat = np.random.random(len(nan)) * (maxx - minx) + minx
        department_temp.loc[nan.index, 'lng'] = lng
        department_temp.loc[nan.index, 'lat'] = lat
    
    department.loc[department['lng'].isnull().values.tolist(), 'lng'] = department_temp['lng']
    department.loc[department['lat'].isnull().values.tolist(), 'lat'] = department_temp['lat']

C:\Users\Ekips\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
